In [1]:
from utils.util import get_project_root
from agents.renderer.test_prompt import prompt,org_text

In [2]:
from utils.llm import LLMProfile
from langchain_core.output_parsers import JsonOutputParser
import json 
final_prompt = prompt.replace("{RAW_TEXT}", org_text)
slice_plan_llm = LLMProfile.chat_bot_llm().invoke(final_prompt).content
slice_plan_json = JsonOutputParser().parse(slice_plan_llm)

In [3]:
slice_plan_json

{'meta': {'title': '강남구 역삼동 리포트',
  'theme': 'dark_gold',
  'renderer': 'google_slides_simple',
  'compat': ['title', 'bluf', 'text', 'table']},
 'slides': [{'type': 'bluf',
   'heading': 'Executive Summary',
   'bullets': ['BLUF: 본 사업지(서울 강남구 역삼동, 전용 84㎡ 120세대)의 권장 분양가 레인지는 3.3㎡당 1,370~1,430만원(총액 약 34.5~36.0억)으로 판단한다. 근거는 1km 내 유사 연식·평형 실거래 평단 1,370~1,420만원/3.3㎡(2023.07~2024.06)와 2km 내 최근 분양가 1,390~1,460만원/3.3㎡(2022.07~2023.09)의 교차 비교 결과이다.',
    '상단 적용(1,430만원/3.3㎡)은 ‘역세권(도보 7~10분 이내)+학세권(대치·역삼 학군 생활권 인접)+브랜드’ 3요건을 충족하고, 금융조건(계약금 10%, 중도금 일부 또는 무이자)을 동반할 때 타당하다. 반대로 3요건 미충족 시 하단(1,370만원/3.3㎡) 또는 그 이하 보수적 책정이 필요하다.',
    '강남구는 최근 5년 미분양 ‘0’(2017.08~2025.08, 2019.05 예외 20호 1개월 내 해소)로 분양 실패 리스크가 구조적으로 낮다(국토교통부 통계누리(국가승인통계)). 다만 가격이 인근 매매 상단을 10% 이상 초과(≈1,500만원/3.3㎡ 이상) 시 단기 미계약 위험이 커진다.',
    '핵심 리스크: (1) 사업지 좌표 기반 입지 데이터(역세권·학세권·생활편의 실보행 거리) 미확보, (2) 2025~2026년 인근 대규모 입주(예: 청담 르엘 1,261세대, 2025.11)로 체감공급 확대, (3) 2025년 9월 누적 강남구 인구 순유출 전환(-477명)로 수요 민감도 상승, (4) 분양가상한제 건축비 고시·심의 변동, (5) 금

In [4]:
# --- 필수: 위반/중복 자동 수정 + 자동 랩핑 ---
def fix_slice_plan(sp: dict, max_lines: int = 22) -> dict:
    def rough_lines(sl):
        n = 0
        n += 1 if sl.get("title") else 0
        n += 1 if sl.get("lead") else 0
        for g in sl.get("groups", []):
            n += 1  # label
            n += len(g.get("items", []))
        n += len(sl.get("sources", []))
        return n

    new_slides = []
    for sl in sp.get("slides", []):
        if sl.get("type") == "text":
            # 1) groups 비어 있으면 '기타' 그룹 생성(lead를 아이템으로)
            groups = sl.setdefault("groups", [])
            if not groups:
                payload = []
                if sl.get("lead"):
                    payload.append(sl["lead"])
                groups.append({"label": "기타", "items": payload or ["내용 요약"]})

            # 2) '출처:'로 시작하는 아이템은 그룹.sources로 이동
            for g in groups:
                items = g.get("items", [])
                keep = []
                for it in items:
                    if isinstance(it, str) and it.strip().startswith("출처:"):
                        g.setdefault("sources", []).append(it.strip().replace("출처:", "", 1).strip())
                    else:
                        keep.append(it)
                g["items"] = keep

            # 3) part 보정
            sl.setdefault("part", 1)

        new_slides.append(sl)
    sp["slides"] = new_slides

    # 4) 자동 랩핑(과다 줄수 분할)
    out = []
    for sl in sp["slides"]:
        if sl.get("type") != "text":
            out.append(sl); continue
        if rough_lines(sl) <= max_lines:
            out.append(sl); continue

        base = {k: v for k, v in sl.items() if k not in ("groups", "part")}
        groups = sl.get("groups", [])
        part = 1
        cur = {"type": "text", **base, "part": part, "groups": []}
        for g in groups:
            cur["groups"].append(g)
            if rough_lines(cur) > max_lines and len(cur["groups"]) > 1:
                cur["groups"].pop()
                out.append(cur)
                part += 1
                cur = {"type": "text", **base, "part": part, "groups": [g]}
        out.append(cur)
    sp["slides"] = out
    return sp




In [5]:
result_json = fix_slice_plan(slice_plan_json)

In [6]:
from agents.renderer.renderer_logic import render_slice_plan
render_slice_plan(result_json)

{'presentationId': '1INYsSVVj8-FBU19D_sHmSzY4PC3fTzQHe__uPL0Yh7s',
 'url': 'https://docs.google.com/presentation/d/1INYsSVVj8-FBU19D_sHmSzY4PC3fTzQHe__uPL0Yh7s/edit'}